# Imports

In [1]:
import os
import gc
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta
import requests
import warnings
warnings.filterwarnings('ignore')

import time
import ast
import json
import numpy as np 
fmp = '6fa0ec6e814bfee671201e723a66b46f' 

def req(r):
    while True:
        try: got = requests.get(r).content
        except: continue
        try: got = json.loads(got.decode('utf-8'))
        except: continue
        if 'Error Message' in got: continue
        return got

import pandas as pd
import os
import gc
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta
import requests
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import pickle
import json
import random
import matplotlib.pyplot as plt

# Collecting Earnings call transcripts

In [2]:
df = pd.DataFrame(req(f'https://financialmodelingprep.com/api/v3/stock/list?apikey={fmp}'))
df = df[(df.exchangeShortName == 'NASDAQ') | (df.exchangeShortName == 'NYSE')]
df = df[df['type'] == 'stock']
df['ticker'] = df.symbol

yer, quat, trans, dtm = [], [], [], []
new = {'ticker':[], 'year': [], 'quarter': [], 'transcript': [], 'time':[]}
for i in (range(len(df))):
    for j in ['2014', '2015', '2016', '2017', '2018', '2019','2020', '2021']:
        for k in ['1', '2', '3', '4']:
            cll = req(f'https://financialmodelingprep.com/api/v3/earning_call_transcript/{df.ticker.iloc[i]}?quarter={k}&year={j}&apikey={fmp}')
            if len(cll) == 0: continue
            else:
                new['ticker'].append(df.ticker.iloc[i])
                new['year'].append(j)
                new['quarter'].append(k)
                new['transcript'].append(cll[0]['content'])
                new['time'].append(cll[0]['date'])
df = pd.DataFrame(new).dropna()
df['time'] = pd.to_datetime(df.time)
df = df[df.time < datetime.today() - timedelta(days=365)] #deleting data for which target is not defined yet 

# Data cleaning

In [3]:
ph = ["hi", "hello", "hey", "good morning", "good day", "good afternoon", "good evening", "greetings", "greeting", "good to see you", "its good seeing you", "how are you", "how're you", "how are you doing", "how ya doin'", "how ya doin", "how is everything", "how is everything going", "how's everything going", "how is you", "how's you", "how are things", "how're things", "how is it going", "how's it going", "how's it goin'", "how's it goin", "how is life been treating you", "how's life been treating you", "how have you been", "how've you been", "what is up", "what's up", "what is cracking", "what's cracking", "what is good", "what's good", "what is happening", "what's happening", "what is new", "what's new", "what is new", "g’day", "howdy", "welcome"]
gc.collect()
import re
def del_txt(x):
    new = []
    for i in x.lower().split('\n')[1:]:
        new_str = ''
        if len(i) == 0: continue
        for j in range(min(100, len(i))):
            if i[j] == ':': break
        if i[:j] == 'operator': continue
        if j != len(i)+1: i = i[j+1:]
        start = 0
        for j in re.finditer('\.|\?|\;', i):
            if i[j.start()] == '?': 
                start = j.start()+1
                continue
            st = i[start:j.start()+1]
            good = True
            for p in ph:
                if re.search(f'{p}[ .,\d]', st) != None:
                    good = False; break
            if st.count(' ') <= 10:
                if re.search(f'thank you[ .,\d]', st) != None: good = False
            if good: new_str += st
            start = j.start()+1
        new.append(new_str)
    return('\n'.join(new))
for i in range(len(df)):
    df.transcript.iloc[i] = del_txt(df.transcript.iloc[i])
gc.collect()

# Stock price collection

**Collecting market capitalization before the earnings call to clear the dataset of small cap stocks**

In [4]:
mass = []
for i in (range(len(df.ticker))):
    prc = req(f"https://financialmodelingprep.com/api/v3/historical-price-full/{df.ticker.iloc[i]}?from={df.time.iloc[i] - timedelta(days = 5)}&to={df.time.iloc[i] - timedelta(days = 1)}&limit=1000000000&apikey={fmp}")
    shrs = req(f"https://financialmodelingprep.com/api/v3/quote/{df.ticker.iloc[i]}?apikey={fmp}")
    if (len(prc) == 0) or (len(shrs) == 0): 
        mass.append(np.nan)
        continue
    shrs = shrs[0]['sharesOutstanding']
    prc = prc['historical'][0]['close']
    mass.append(prc*shrs/1000000000)
df['mcap'] = mass

**Collecting close price of the stock in one day after the earnings call and in one year after and then calculating difference over this period**

In [5]:
diff = []
df['time'] = pd.to_datetime(df.time)
for i in (range(0, len(df))):
    prc = req(f'https://financialmodelingprep.com/api/v3/historical-price-full/{df.ticker.iloc[i]}?from={df.time.iloc[i] + timedelta(days = 1)}&to={df.time.iloc[i] + timedelta(days = 365)}&apikey={fmp}')
    if len(prc) == 0:
        diff.append(np.nan)
        continue
    con = prc['historical'][0]['close']
    prc = prc['historical'][-1]['close']
    if prc == 0:
        diff.append(np.nan)
        continue
    diff.append(con/prc)
df['change'] = diff

In [6]:
oov_tok = "<OOV>"
tokenizer = Tokenizer(oov_token=oov_tok)
tokenizer.fit_on_texts(transcripts.transcript)

In [7]:
df = df[df.mcap > 1]
df = df[df.change < 5] #cleaning from outliers with very high growth

## Positivity

In [8]:
def fit(train, valid, embedding_dim, max_length, trunc_type='post', padding_type='post', oov_tok = "<OOV>"):
    vocab_size = len(tokenizer.word_index) + 1
    embedding_dim = embedding_dim
    max_length = max_length

    training_sequences = tokenizer.texts_to_sequences(train.transcript)
    training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

    testing_sequences = tokenizer.texts_to_sequences(valid.transcript)
    testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    os.environ['PYTHONHASHSEED']=str(1)
    tf.random.set_seed(1)
    np.random.seed(1)
    random.seed(1)

    y_train = np.array(train.y)
    y_test = np.array(valid.y)

    num_epochs = 3
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=10000),
        tf.keras.layers.Conv1D(20, 2),
        tf.keras.layers.GlobalAveragePooling1D(),
        tf.keras.layers.Dense(200, activation='relu'),
        tf.keras.layers.Dense(100, activation='relu'),
        tf.keras.layers.Dense(50, activation='relu'),
        tf.keras.layers.Dense(24, activation='relu'),
        tf.keras.layers.Dense(12, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC'])
    model.fit(training_padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test), verbose=0)
    return model.predict(testing_padded)

In [9]:
mod, rand, year = [], [], []
preds_pos = pd.DataFrame()
for i in range(2016, 2022):
    train = df[(df.year.astype(int) < i)]
    valid = df[df.year.astype(int) == i]
    tar = 1.0
    train['y'] = 0
    train['y'][train.change >= tar] = 1
    valid['y'] = 0
    valid['y'][valid.change >= tar] = 1
    valid['proba'] = fit(train, valid, embedding_dim = 16, max_length = 10000)
    preds_pos = preds_pos.append(valid.drop(columns='transcript').sort_values(by = 'proba', ascending=False).iloc[range(0,int(0.2*len(valid)))])
    mod.append(round(valid.sort_values(by = 'proba', ascending=False).iloc[range(0,int(0.2*len(valid)))].change.mean(), 2))
    rand.append(round(valid.change.mean(), 2))
    year.append(i)

In [10]:
pd.DataFrame({'Year': year, 'Model Profit': mod, 'Market Profit': rand}).sort_values(by = 'Year')

,Year,Model Profit,Market Profit
0,2016,1.36,1.17
1,2017,1.27,1.07
2,2018,1.18,1.02
3,2019,1.24,1.04
4,2020,1.51,1.47
5,2021,1.43,1.35


In [11]:
mod, rand, year = [], [], []
preds = pd.DataFrame()
for i in range(2016, 2022):
    train = df[(df.year.astype(int) < i)]
    valid = df[df.year.astype(int) == i]
    tar = 0.9
    train['y'] = 0
    train['y'][train.change <= tar] = 1
    valid['y'] = 0
    valid['y'][valid.change <= tar] = 1
    valid['proba'] = fit(train, valid, embedding_dim = 16, max_length = 10000)
    preds = preds.append(valid.drop(columns='transcript').sort_values(by = 'proba', ascending=False).iloc[range(0,int(0.2*len(valid)))])
    mod.append(round(valid.sort_values(by = 'proba', ascending=False).iloc[range(0,int(0.2*len(valid)))].change.mean(), 2))
    year.append(i)

In [12]:
pd.DataFrame({'Year': year, 'Model Profit': mod}).sort_values(by = 'Year')

,Year,Model Profit
0,2016,0.85
1,2017,0.83
2,2018,0.74
3,2019,0.79
4,2020,1.01
5,2021,0.91


## Extracting key quotes

In [13]:
!pip install lime > /dev/null 2> /dev/null

In [14]:
train = df[(df.year.astype(int) < 2021)]
valid = df[df.year.astype(int) == 2021]
tar = 1
train['y'] = 0
train['y'][train.change >= tar] = 1
valid['y'] = 0
valid['y'][valid.change >= tar] = 1
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16
max_length = 10000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_sequences = tokenizer.texts_to_sequences(train.transcript)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

testing_sequences = tokenizer.texts_to_sequences(valid.transcript)
testing_padded = pad_sequences(testing_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
os.environ['PYTHONHASHSEED']=str(1)
tf.random.set_seed(1)
np.random.seed(1)
random.seed(1)

y_train = np.array(train.y)
y_test = np.array(valid.y)

num_epochs = 3
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=10000),
    tf.keras.layers.Conv1D(20, 2),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(200, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['AUC'])
model.fit(training_padded, y_train, epochs=num_epochs, validation_data=(testing_padded, y_test), verbose=0)

In [15]:
from sklearn.pipeline import make_pipeline
def predictor(transcript):
    new_s = tokenizer.texts_to_sequences(pd.Series(transcript)) 
    new_p =  pad_sequences(new_s, maxlen=max_length, padding=padding_type, truncating=trunc_type)
    pr = model.predict(new_p)
    return np.concatenate([(1-pr.reshape(1,-1)).reshape(-1,1), pr], 1)

In [16]:
from lime.lime_text import LimeTextExplainer
idx = 559


explainer = LimeTextExplainer(class_names=['Not Grew', 'Grew'])
exp = explainer.explain_instance(valid.transcript.loc[idx], predictor, num_samples= 1000, num_features= 10000)

In [17]:
exp.show_in_notebook(text=True)

In [18]:
import re
wlist = exp.as_list()
weights, sents = [-np.inf]*3, [-np.inf]*3
trans = valid.transcript.loc[idx].replace('u.s.', 'US').split('.')
for _, i in (enumerate(trans)):
    if len(i) < 10: continue
    weight = 0
    for j in wlist:
        weight += j[1]*len(re.findall(f'\\b{j[0]}\\b', i))
    ll = i.count(' ') + 1
    weight /= ll
    mi = np.argmin(weights)
    if weight > weights[mi]:
        weights[mi] = weight
        if ll > 7:
            sents[mi] = i
        else:
            sents[mi] = trans[_-1] + i
weights, sents

([0.005754602070658243, 0.012240548799944594, 0.010493718681927748],
 [' our teams have navigated with agility and focus to recalibrate supply and demand, to increase digital distribution capacity, to secure liquidity and to tightly manage costs, all while ensuring the health and safety of our employees and consumers as a result, nike, inc',
  ' now let’s turn to the details of our first quarter financial results and operating segment performance nike, inc',
  ' sustainability will continue to be a key aspect of our innovation agenda going forward this is how nike leads'])

## Performance visualization

In [19]:
dic = {'Year':[], 'Small':[], 'Medium':[], 'Large':[], 'Severe':[]}
for i in preds.year.unique():
    yr = preds[preds.year == i]
    new = yr.dropna().sort_values(by = 'negativity').iloc[range(round(0.2*len(yr)))]
    new = new[new.change < 1]
    dic['Year'].append(i)
    dic['Small'].append(len(new[new.change > 0.9])/len(new))
    dic['Medium'].append(len(new[(new.change > 0.75) & (new.change <= 0.9)])/len(new))
    dic['Large'].append(len(new[(new.change > 0.5) & (new.change <= 0.75)])/len(new))
    dic['Severe'].append(len(new[(new.change <= 0.5)])/len(new))

In [20]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.express as px
df = pd.DataFrame(dic)
fig = make_subplots(rows=2, cols=3, specs=[[{"type": "pie"}]*3, [{"type": "pie"}]*3])
names = ['Small', 'Medium', 'Large', 'Severe']
j, k = 1, 0
for i in df.Year:
    if k == 3:
        j+=1
        k = 0
    k+=1

    fig.add_trace(   go.Pie(labels=names, values=list(df[df.Year == i][names].values[0]), title = i ), row=j, col = k)
    fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(title = 'Decline distributions of stocks with similar negativity')
fig.show()

In [21]:
dic = {'Year':[], 'Small':[], 'Medium':[], 'Large':[], 'Severe':[]}
for i in preds.year.unique():
    yr = preds[preds.year == i]
    new = yr.dropna()
    new = new[new.change < 1]
    dic['Year'].append(i)
    dic['Small'].append(len(new[new.change > 0.9])/len(new))
    dic['Medium'].append(len(new[(new.change > 0.75) & (new.change <= 0.9)])/len(new))
    dic['Large'].append(len(new[(new.change > 0.5) & (new.change <= 0.75)])/len(new))
    dic['Severe'].append(len(new[(new.change <= 0.5)])/len(new))

In [22]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import plotly.express as px
df = pd.DataFrame(dic)
fig = make_subplots(rows=2, cols=3, specs=[[{"type": "pie"}]*3, [{"type": "pie"}]*3])
names = ['Small', 'Medium', 'Large', 'Severe']
j, k = 1, 0
for i in df.Year:
    if k == 3:
        j+=1
        k = 0
    k+=1

    fig.add_trace(   go.Pie(labels=names, values=list(df[df.Year == i][names].values[0]), title = i ), row=j, col = k)
    fig.update_traces(hole=.4, hoverinfo="label+percent")

fig.update_layout(title = 'Decline distributions of the market')
fig.show()

In [23]:
years, avg, nf = [], [], []
for i in preds_pos.sort_values(by = 'year').year.unique():
    nec = preds_pos[preds_pos.year == i]
    years.append(i)
    avg.append(np.sum((nec.proba < 0.9) & (nec.proba >= 0.85) & (nec.change > 1)) / np.sum((nec.proba < 0.9) & (nec.proba >= 0.85)))
    nf.append(np.sum(nec.change > 1)/len(nec))

In [24]:
import plotly.express as px
bar = pd.DataFrame({'info': years, 'Stocks with high positivity': avg, 'Market': nf})

fig = px.bar(bar, x='info', y=['Market', 'Stocks with high positivity'], barmode="group", labels={'value':'Percentage'}, hover_data={'info':False, # remove species from hover data
                             'variable':False})
fig.update_layout(yaxis= { 'tickformat': ',.0%'})
fig.update_layout( autosize=False, width=580, height=580)
fig.update_xaxes(title = '')
fig.update_layout(legend_title_text='')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(template='plotly_white')
fig.update_layout(legend=dict(
    orientation="h",
    yanchor="bottom",
    y=1.02,
    xanchor='auto',
    x=0.2
))
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=73e8da85-b373-4448-aa84-bad98fe8921d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>